In [44]:
import pandas as pd
import numpy as np
import tensorflowjs as tfjs

In [45]:
df = pd.read_csv("HR_comma_sep.csv")

In [46]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


# Creating dummy variables of categorical columns

In [47]:
feats = ['department','salary']
df_final = pd.get_dummies(df,columns=feats,drop_first=True)

In [48]:
df_final.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department_RandD,department_accounting,department_hr,department_management,department_marketing,department_product_mng,department_sales,department_support,department_technical,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,0,0,0,0,1,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,0,0,0,0,1,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,0,0,0,0,1,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,0,0,0,0,1,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,0,0,0,0,1,0,0,1,0


In [49]:
df_final.shape

(14999, 19)

# Splitting Testing and Training Data

In [50]:
from sklearn.model_selection import train_test_split

// Droping left column (target variable)

In [51]:
X = df_final.drop(['left'],axis=1).values
y = df_final['left'].values

// 70-30 split of training and testing data

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Transforming the data

// Scaling the data set (mean=0, normally distributed)

In [53]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Building the Artificial Neural Network

In [54]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [55]:
classifier = Sequential()

// Adding Layers

In [56]:
classifier.add(Dense(9, kernel_initializer = "uniform",activation = "relu", input_dim=18))

// Adding Output Layer

In [57]:
classifier.add(Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))

// Applying Gradient Descent (adam optimization)

In [58]:
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

// Fitting dataset to classifier

In [59]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)

Epoch 1/10
1050/1050 [==============================] - 3s 2ms/step - loss: 0.4176 - accuracy: 0.7948
Epoch 2/10
1050/1050 [==============================] - 2s 2ms/step - loss: 0.2714 - accuracy: 0.8461
Epoch 3/10
1050/1050 [==============================] - 3s 3ms/step - loss: 0.2246 - accuracy: 0.9243
Epoch 4/10
1050/1050 [==============================] - 3s 3ms/step - loss: 0.1992 - accuracy: 0.9489
Epoch 5/10
1050/1050 [==============================] - 2s 2ms/step - loss: 0.1849 - accuracy: 0.9533
Epoch 6/10
1050/1050 [==============================] - 3s 2ms/step - loss: 0.1756 - accuracy: 0.9541
Epoch 7/10
1050/1050 [==============================] - 3s 2ms/step - loss: 0.1697 - accuracy: 0.9549
Epoch 8/10
1050/1050 [==============================] - 3s 2ms/step - loss: 0.1660 - accuracy: 0.9557
Epoch 9/10
1050/1050 [==============================] - 3s 3ms/step - loss: 0.1629 - accuracy: 0.9561
Epoch 10/10
1050/1050 [==============================] - 3s 3ms/step - loss: 0.161

# Running Predictions on the Test Set

In [60]:
y_pred = classifier.predict(X_test)

141/141 [==============================] - 0s 2ms/step


In [61]:
y_pred = (y_pred > 0.5)

# Checking the Confusion Matrix

In [62]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[3320,   97],
       [ 102,  981]], dtype=int64)

// correct predictions: 3321+992=4313
//  total observations: 3321+108+79+992=4500
//  accuracy=4313/4500=95.8%

# Making a single prediction

In [71]:
new_pred = classifier.predict(sc.transform(np.array([[0.26,0.7 ,3., 238., 6., 0.,0.,0.,0., 0.,0.,0.,0.,0.,1.,0., 0.,1.]])))

1/1 [==============================] - 0s 77ms/step


In [81]:
test_pred = classifier.predict(sc.transform(np.array([[0.10,0.2 ,1, 100, 2, 1,0,1,0, 1,0,1,0,1,1,1, 0,1]])))

1/1 [==============================] - 0s 48ms/step


In [84]:
sc.transform(np.array([[0.10,0.2 ,1, 100, 2, 1,0,1,0, 1,0,1,0,1,1,1, 0,1]]))

array([[-2.05449089, -3.03951905, -2.27040209, -2.02383327, -1.01757982,
         2.43422713, -0.14798629,  4.21286296, -0.23534659,  4.4120162 ,
        -0.20939364,  4.05108343, -0.25478482,  1.63196011,  2.38301347,
         2.13493962, -0.97711669,  1.15595246]])

In [83]:
test_pred

array([[0.7203589]], dtype=float32)

In [72]:
new_pred

array([[0.01444816]], dtype=float32)

In [64]:
new_pred = (new_pred > 0.5)
new_pred

array([[False]])

// saving model

In [70]:
tfjs.converters.save_keras_model(classifier, 'C:/Users/aniru/DeepLearningModelMinorProject')

# Improving the Model Accuracy

// K-fold cross-validation

In [27]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

// Function to be passed to kerasClassifier
// this function creates the neural network design that we used earlier

In [28]:
def make_classifier():
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [29]:
classifier = KerasClassifier(build_fn = make_classifier, batch_size=10, nb_epoch=1)

C:\Users\aniru\AppData\Local\Temp\ipykernel_15816\128404206.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = make_classifier, batch_size=10, nb_epoch=1)


In [30]:
accuracies = cross_val_score(estimator = classifier,X = X_train,y = y_train,cv = 10,n_jobs = -1)

In [31]:
mean = accuracies.mean()
mean

0.8353247046470642

In [32]:
variance = accuracies.var()
variance

0.0033742223006562712

# Adding Dropout Regularization (to fight Over-Fitting)

Adding dropout layer

In [33]:
from keras.layers import Dropout

classifier = Sequential()
classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
classifier.add(Dropout(rate = 0.1))
classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

# Hyperparameter Tuning

Grid search

In [34]:
from sklearn.model_selection import GridSearchCV
def make_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= optimizer,loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [35]:
classifier = KerasClassifier(build_fn = make_classifier)

C:\Users\aniru\AppData\Local\Temp\ipykernel_15816\4215114608.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn = make_classifier)


Setting up parameters to experiment with

In [36]:
params = {
    'batch_size':[20,35],
    'epochs':[2,3],
    'optimizer':['adam','rmsprop']
}

In [37]:
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=params,
                           scoring="accuracy",
                           cv=2)

In [38]:
grid_search = grid_search.fit(X_train,y_train)

Epoch 1/2
263/263 [==============================] - 1s 2ms/step - loss: 0.5715 - accuracy: 0.7666
Epoch 2/2
165/165 [==============================] - 0s 2ms/step
Epoch 1/2
263/263 [==============================] - 2s 3ms/step - loss: 0.5854 - accuracy: 0.7579
Epoch 2/2
165/165 [==============================] - 0s 2ms/step
Epoch 1/2
263/263 [==============================] - 1s 2ms/step - loss: 0.5831 - accuracy: 0.7670
Epoch 2/2
165/165 [==============================] - 0s 2ms/step
Epoch 1/2
263/263 [==============================] - 1s 2ms/step - loss: 0.5887 - accuracy: 0.7619
Epoch 2/2
165/165 [==============================] - 0s 2ms/step
Epoch 1/3
263/263 [==============================] - 1s 2ms/step - loss: 0.5895 - accuracy: 0.7737
Epoch 2/3
263/263 [==============================] - 1s 2ms/step - loss: 0.4084 - accuracy: 0.8066
Epoch 3/3
165/165 [==============================] - 0s 2ms/step
Epoch 1/3
263/263 [==============================] - 1s 2ms/step - loss: 0.5871 -

In [39]:
best_param = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [40]:
best_param

{'batch_size': 20, 'epochs': 3, 'optimizer': 'rmsprop'}

In [41]:
best_accuracy

0.8665574939444247